# Imports

In [1]:
import numpy as np
import requests as rq
from scipy.optimize import linprog

# Query Function
*From template*

In [2]:
# Retrieve answer to challenge for a given query
def query(challenge_id, query_vector, submit=False):
    # Only alphanumeric challenge_id and vextor entries in {-1,+1} are allowed:
    assert(challenge_id.isalnum())
    assert(np.max(np.minimum(np.abs(query_vector-1),np.abs(query_vector+1))) == 0)

    # if query array is 1d, make it 2d
    if query_vector.ndim == 1:
        query_vector = query_vector.reshape(1,-1)

    payload = {
        'challengeid': challenge_id,
        'submit': submit,
        'query': str(query_vector.tolist())
    }
    response = rq.post("https://rasmuspagh.pythonanywhere.com/query", data = payload).json()
    if submit == False:
        return np.array(eval(response['result']))
    else:
        return response['result']

# Config

In [3]:
challenge_id = 'prs901third' # identifier for hidden dataset
n = 256 # number of entries in hidden dataset
num_queries = 2*n # number of queries to be asked

# Generating Queries

In [4]:
queries = np.random.choice([-1,+1], size=(num_queries,n)) # Set of random queries

# Querying

In [5]:
query_results = query(challenge_id, queries)

# Optimization

In [6]:
optimized_result = linprog(
    - query_results @ queries,
    A_ub=queries,
    b_ub=np.abs(query_results),
    bounds=(-1,1)
)

# Formatting Final Guess

In [7]:
guess = optimized_result.x
guess[guess >= 0] = 1
guess[guess < 0] = -1

# Submission

In [8]:
best_query_result = query(challenge_id, guess, submit=True)
print(best_query_result)

156


# Result

In [9]:
print(f"\nReconstruction attack achieves fraction {round((1 + best_query_result / n) / 2, 2)} correct values")


Reconstruction attack achieves fraction 0.8 correct values
